In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime



# Charger le dataset
df = pd.read_csv("data.csv", encoding='ISO-8859-1')

# Afficher les premières lignes
df.head()



# Vérifier les dimensions du dataset
print(f"Nombre de lignes : {df.shape[0]}")
print(f"Nombre de colonnes : {df.shape[1]}")

# Aperçu des colonnes et types de données
df.info()

# Statistiques de base sur les variables numériques
df.describe()


# Vérifier les valeurs manquantes
df.isnull().sum()

#  Supprimer les valeurs manquantes

df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['CustomerID'] = df['CustomerID'].astype(str)

# Analyse descriptive
#  Produits les plus vendus
df['Description'].value_counts().head(10)

# CA total par pays
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False)

# Clients les plus rentables
df.groupby('CustomerID')['TotalPrice'].sum().sort_values(ascending=False).head(10)



# Visualisation des tendances

#  Distribution des ventes par pays

plt.figure(figsize=(12, 5))
sns.barplot(y=df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).index,
            x=df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).values)
plt.xlabel("Chiffre d'affaires")
plt.ylabel("Pays")
plt.title("CA par pays")
plt.show()


# Évolution des ventes dans le temps

df.set_index('InvoiceDate').resample('M')['TotalPrice'].sum().plot(figsize=(12, 5))
plt.title("Évolution des ventes mensuelles")
plt.show()

# Top 10 des produits les plus vendus

plt.figure(figsize=(12, 5))
sns.barplot(x=df['Description'].value_counts().head(10).values,
            y=df['Description'].value_counts().head(10).index)
plt.title("Top 10 des produits les plus vendus")
plt.xlabel("Nombre de ventes")
plt.ylabel("Produits")
plt.show()


# Analyse des performances des produits

# Produits les plus rentables

top_products = df.groupby('Description')['TotalPrice'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(12, 5))
sns.barplot(x=top_products.values, y=top_products.index)
plt.title("Top 10 des produits les plus rentables")
plt.xlabel("Revenu total")
plt.ylabel("Produits")
plt.show()



#  Analyse des retours produits

df['Retour'] = df['Quantity'] < 0  # Si quantité < 0, c'est un retour
df['Retour'].value_counts()


# Taux de retour par produit

retours = df[df['Retour']].groupby('Description')['Quantity'].sum().sort_values()
plt.figure(figsize=(12, 5))
sns.barplot(x=retours.values[:10], y=retours.index[:10])
plt.title("Top 10 des produits les plus retournés")
plt.xlabel("Nombre de retours")
plt.ylabel("Produits")
plt.show()


# Analyse des clients et segmentation
# Fréquence des achats par client

client_orders = df.groupby('CustomerID')['InvoiceNo'].nunique().sort_values(ascending=False)
plt.figure(figsize=(12, 5))
sns.histplot(client_orders, bins=50, kde=True)
plt.title("Distribution du nombre d'achats par client")
plt.xlabel("Nombre d'achats")
plt.ylabel("Nombre de clients")
plt.show()


# Valeur moyenne des commandes

df.groupby('CustomerID')['TotalPrice'].mean().hist(bins=50)
plt.title("Valeur moyenne des commandes par client")
plt.xlabel("Valeur moyenne")
plt.ylabel("Nombre de clients")
plt.show()



# Calcul des scores RFM
# Date de référence (dernière date du dataset)
date_ref = df['InvoiceDate'].max()

# Calcul des RFM
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (date_ref - x.max()).days,  # Récence
    'InvoiceNo': 'count',  # Fréquence
    'TotalPrice': 'sum'  # Monétaire
})

# Renommer les colonnes
rfm.rename(columns={'InvoiceDate': 'Recence', 'InvoiceNo': 'Frequence', 'TotalPrice': 'Monetaire'}, inplace=True)

# Afficher un aperçu
rfm.head()


# Catégorisation en groupes


# Fonction pour catégoriser en 4 segments (1 = moins important, 4 = très important)
rfm['R_Score'] = pd.qcut(rfm['Recence'], q=4, labels=[4, 3, 2, 1])
rfm['F_Score'] = pd.qcut(rfm['Frequence'], q=4, labels=[1, 2, 3, 4])
rfm['M_Score'] = pd.qcut(rfm['Monetaire'], q=4, labels=[1, 2, 3, 4])

# Score final RFM = R + F + M
rfm['RFM_Score'] = rfm[['R_Score', 'F_Score', 'M_Score']].astype(int).sum(axis=1)

# Segmentation des clients
rfm['Segment'] = rfm['RFM_Score'].apply(lambda x: 'VIP' if x >= 10 else 'Fidèle' if x >= 7 else 'Occasionnel')

# Visualiser les segments
rfm['Segment'].value_counts()

# Visualisation des segments

plt.figure(figsize=(8, 5))
sns.countplot(x=rfm['Segment'], palette='viridis')
plt.title("Répartition des clients par segment")
plt.show()



# Création d'une série temporelle des ventes mensuelles
df_monthly = df.set_index('InvoiceDate').resample('M')['TotalPrice'].sum()

# Visualisation
plt.figure(figsize=(12, 5))
df_monthly.plot()
plt.title("Évolution du chiffre d'affaires mensuel")
plt.xlabel("Date")
plt.ylabel("CA")
plt.show()






